In [29]:
from typing import Set, List

import pandas as pd
from tqdm.auto import tqdm
import re
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

# Loading texts

In [30]:
number_finding_regex = re.compile(r'\d+')
def get_ids(str_with_ids: str) -> List[int]:
    return number_finding_regex.findall(str_with_ids)

In [31]:
df_texts_part_1 = pd.read_csv('texts/100000txtPART1.csv', sep=';',).dropna()
df_texts_part_2 = pd.read_csv('texts/100000txtPART2.csv', sep=';').dropna()
df_texts_part_3 = pd.read_csv('texts/25000txt.csv', sep=';').dropna()
df_texts_part_4 = pd.read_csv('texts/2000txt.csv', sep=';').dropna()
df_texts_full = pd.concat([df_texts_part_1, df_texts_part_2, df_texts_part_3, df_texts_part_4], ignore_index=True)


new_df_data = []
for _, row in tqdm(df_texts_full.iterrows(), total=len(df_texts_full)):
    track_ids = get_ids(row.trackIds)

    for track_id in track_ids:
        new_df_data.append([
            row.resource,
            int(track_id),
        ])

df_texts_full = pd.DataFrame(
    new_df_data,
    columns=['text', 'track_id'],
)
df_texts_full.drop_duplicates(inplace=True, subset=['track_id'])

print(df_texts_full.info())
df_texts_full.head()

  0%|          | 0/91397 [00:00<?, ?it/s]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 172029 entries, 0 to 413793
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   text      172029 non-null  object
 1   track_id  172029 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.9+ MB
None


,text,track_id
0,"Вновь о том, что день уходит с земли в час веч...",17869014
1,"Вновь о том, что день уходит с земли в час веч...",24264852
2,"Вновь о том, что день уходит с земли в час веч...",74175278
3,Вот и осталось\nЛишь снять усталость\nИ этот в...,5335355
4,Вот и осталось\nЛишь снять усталость\nИ этот в...,35903122


# Loading meta

In [32]:
raw_meta = pd.read_csv('meta/100000meta.csv', sep=';')
raw_meta.drop_duplicates(inplace=True, subset=['track_id'])
raw_meta.drop(inplace=True, columns=[
    'added', 'albums', 'artist_composer', 'artist_composer_composed',
    'artist_performer', 'artist_performer_composed', 'artists', 'artistsName',
    'artists_composed', 'clips', 'content_kind', 'duration', 'explicit',
    'external_ids', 'genre_ids', 'genres', 'is_russian', 'isrc', 'language',
    'lyrics', 'main_album_id', 'main_artist', 'main_artist_composed',
    'majors', 'meta_genre_ids', 'meta_genres', 'non_original',
    'rating_position', 'rights_regions_geoids', 'rights_regions_info',
    'start_date', 'storage_dir_rnd_part', 'stream_30', 'streamers_30',
])
raw_meta.version = raw_meta.version.fillna('')

print(raw_meta.info())
raw_meta.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   other_titles  100000 non-null  object 
 1   rating        99807 non-null   float64
 2   title         100000 non-null  object 
 3   track_id      100000 non-null  int64  
 4   version       100000 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 4.6+ MB
None


,other_titles,rating,title,track_id,version
0,[],961791.0,Ночью на кухне,108205536,
1,[],852844.0,Позвони,107314429,
2,[],643184.0,Иностранец,107480583,
3,[],1023430.0,По барам,104964266,
4,[],865433.0,Стрелы,106246540,


In [33]:
cover_regex = re.compile(r'\b(cover|remix)\b', re.IGNORECASE)

df_meta = pd.DataFrame({
    'track_id': raw_meta.track_id,
    'rating': raw_meta.rating.apply(lambda x: int(x) if pd.notna(x) else 0),
    'rating_nan': raw_meta.rating.apply(pd.isna),
    'title_regex_flag': raw_meta.title.apply(lambda x: bool(cover_regex.match(x))),
    'other_titles_regex_flag': raw_meta.other_titles.apply(lambda x: bool(cover_regex.match(x))),
    'version_regex_flag': raw_meta.version.apply(lambda x: bool(cover_regex.match(x))),
})

print(df_meta.info())
df_meta.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 6 columns):
 #   Column                   Non-Null Count   Dtype
---  ------                   --------------   -----
 0   track_id                 100000 non-null  int64
 1   rating                   100000 non-null  int64
 2   rating_nan               100000 non-null  bool 
 3   title_regex_flag         100000 non-null  bool 
 4   other_titles_regex_flag  100000 non-null  bool 
 5   version_regex_flag       100000 non-null  bool 
dtypes: bool(4), int64(2)
memory usage: 2.7 MB
None


,track_id,rating,rating_nan,title_regex_flag,other_titles_regex_flag,version_regex_flag
0,108205536,961791,False,False,False,False
1,107314429,852844,False,False,False,False
2,107480583,643184,False,False,False,False
3,104964266,1023430,False,False,False,False
4,106246540,865433,False,False,False,False


In [34]:
main_df = pd.read_csv('ymtop2000.csv', sep=';').dropna(how='all', axis=1)
print(main_df.info())
main_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   rank         2000 non-null   int64 
 1   artistId     1997 non-null   object
 2   artistsName  1997 non-null   object
 3   trackTitle   1997 non-null   object
 4   track_id     2000 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 78.2+ KB
None


,rank,artistId,artistsName,trackTitle,track_id
0,1,4611844,Miyagi & Эндшпиль,Saloon,108315318
1,2,4611844,Miyagi & Эндшпиль,Silhouette,108315330
2,3,15691816,ANNA ASTI,Ночью на кухне,108205536
3,4,3719594,Jah Khalib,Осенняя,108284286
4,5,6098375,GOARTUR,Иностранец,107480583


In [35]:
train_df = main_df.merge(df_texts_full, left_on='track_id', right_on='track_id', how='left')
train_df = train_df.merge(df_meta, left_on='track_id', right_on='track_id', how='left')
print(train_df.info())
train_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 0 to 1999
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   rank                     2000 non-null   int64 
 1   artistId                 1997 non-null   object
 2   artistsName              1997 non-null   object
 3   trackTitle               1997 non-null   object
 4   track_id                 2000 non-null   int64 
 5   text                     1446 non-null   object
 6   rating                   2000 non-null   int64 
 7   rating_nan               2000 non-null   bool  
 8   title_regex_flag         2000 non-null   bool  
 9   other_titles_regex_flag  2000 non-null   bool  
 10  version_regex_flag       2000 non-null   bool  
dtypes: bool(4), int64(3), object(4)
memory usage: 132.8+ KB
None


,rank,artistId,artistsName,trackTitle,track_id,text,rating,rating_nan,title_regex_flag,other_titles_regex_flag,version_regex_flag
0,1,4611844,Miyagi & Эндшпиль,Saloon,108315318,В салуне злого вестерна я заливал печаль\nНас ...,808778,False,False,False,False
1,2,4611844,Miyagi & Эндшпиль,Silhouette,108315330,О-уо-о-уо\nОу-оу-оу-о-о\n\nКрадётся лёгкий сил...,852296,False,False,False,False
2,3,15691816,ANNA ASTI,Ночью на кухне,108205536,"Она не боится темноты, как в детстве\nБольше н...",961791,False,False,False,False
3,4,3719594,Jah Khalib,Осенняя,108284286,В сером небе птицы улетят\nЗастучит опять что-...,441685,False,False,False,False
4,5,6098375,GOARTUR,Иностранец,107480583,Подари мне первый танец\nЗабери меня с собой\n...,643184,False,False,False,False


In [36]:
toloka_results_1 = pd.read_csv('Toloka_results/dawid_skene_1.tsv', sep='\t')
toloka_results_2 = pd.read_csv('Toloka_results/dawid_skene_2.tsv', sep='\t')
toloka_results = pd.concat([toloka_results_1, toloka_results_2], ignore_index=True).dropna(how='all', axis=1)
toloka_results.drop_duplicates(inplace=True, subset=['INPUT:track'])
print(toloka_results.info())
toloka_results.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1994 entries, 0 to 2017
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   INPUT:name         1994 non-null   object
 1   INPUT:playlist-1   1994 non-null   object
 2   INPUT:playlist-2   1994 non-null   object
 3   INPUT:playlist-3   1994 non-null   object
 4   INPUT:track        1994 non-null   object
 5   OUTPUT:result      1994 non-null   object
 6   CONFIDENCE:result  1994 non-null   object
dtypes: object(7)
memory usage: 124.6+ KB
None


,INPUT:name,INPUT:playlist-1,INPUT:playlist-2,INPUT:playlist-3,INPUT:track,OUTPUT:result,CONFIDENCE:result
0,Ваня Дмитриенко.31-я весна,https://music.yandex.ru/users/music-blog/playl...,https://music.yandex.ru/users/music.partners/p...,https://music.yandex.ru/users/music-blog/playl...,https://music.yandex.ru/track/108470485,yes,99.99%
1,ANNA ASTI.По барам,https://music.yandex.ru/users/music-blog/playl...,https://music.yandex.ru/users/music.partners/p...,https://music.yandex.ru/users/music-blog/playl...,https://music.yandex.ru/track/104964266,no,99.95%
2,"Yves V, Ilkay Sencan, Emie.Not So Bad",https://music.yandex.ru/users/music-blog/playl...,https://music.yandex.ru/users/music.partners/p...,https://music.yandex.ru/users/music-blog/playl...,https://music.yandex.ru/track/61288362,yes,99.99%
3,"CHEBANOV.Люби меня, люби",https://music.yandex.ru/users/music-blog/playl...,https://music.yandex.ru/users/music.partners/p...,https://music.yandex.ru/users/music-blog/playl...,https://music.yandex.ru/track/109274714,yes,100.00%
4,"Markul, Тося Чайкина.Стрелы",https://music.yandex.ru/users/music-blog/playl...,https://music.yandex.ru/users/music.partners/p...,https://music.yandex.ru/users/music-blog/playl...,https://music.yandex.ru/track/106246540,no,99.99%


In [37]:
def get_track_id(track_path: str) -> int:
    return int(track_path.split('/')[-1])

In [38]:
toloka_results['track_id'] = toloka_results['INPUT:track'].apply(get_track_id)
toloka_results.drop(
    columns=['INPUT:playlist-1', 'INPUT:playlist-2', 'INPUT:playlist-3', 'INPUT:track'],
    inplace=True,
)
toloka_results.drop_duplicates(inplace=True, subset=['track_id'])
print(toloka_results.info())
toloka_results.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1994 entries, 0 to 2017
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   INPUT:name         1994 non-null   object
 1   OUTPUT:result      1994 non-null   object
 2   CONFIDENCE:result  1994 non-null   object
 3   track_id           1994 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 77.9+ KB
None


,INPUT:name,OUTPUT:result,CONFIDENCE:result,track_id
0,Ваня Дмитриенко.31-я весна,yes,99.99%,108470485
1,ANNA ASTI.По барам,no,99.95%,104964266
2,"Yves V, Ilkay Sencan, Emie.Not So Bad",yes,99.99%,61288362
3,"CHEBANOV.Люби меня, люби",yes,100.00%,109274714
4,"Markul, Тося Чайкина.Стрелы",no,99.99%,106246540


In [39]:
main_df[~main_df.track_id.isin(toloka_results.track_id)]

,rank,artistId,artistsName,trackTitle,track_id
0,1,4611844,Miyagi & Эндшпиль,Saloon,108315318
2,3,15691816,ANNA ASTI,Ночью на кухне,108205536
3,4,3719594,Jah Khalib,Осенняя,108284286
6,7,6229247,GAYAZOV$ BROTHER$,ФАИНА,107948955
42,43,6467364,NЮ,Некуда бежать,107058193
659,660,NaN,NaN,NaN,107429485
860,861,NaN,NaN,NaN,107446873
1430,1431,NaN,NaN,NaN,27198553


In [40]:
train_df = train_df.merge(toloka_results, left_on='track_id', right_on='track_id', how='left')
print(train_df.info())
train_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 0 to 1999
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   rank                     2000 non-null   int64 
 1   artistId                 1997 non-null   object
 2   artistsName              1997 non-null   object
 3   trackTitle               1997 non-null   object
 4   track_id                 2000 non-null   int64 
 5   text                     1446 non-null   object
 6   rating                   2000 non-null   int64 
 7   rating_nan               2000 non-null   bool  
 8   title_regex_flag         2000 non-null   bool  
 9   other_titles_regex_flag  2000 non-null   bool  
 10  version_regex_flag       2000 non-null   bool  
 11  INPUT:name               1992 non-null   object
 12  OUTPUT:result            1992 non-null   object
 13  CONFIDENCE:result        1992 non-null   object
dtypes: bool(4), int64(3), object(7)
memory u

,rank,artistId,artistsName,trackTitle,track_id,text,rating,rating_nan,title_regex_flag,other_titles_regex_flag,version_regex_flag,INPUT:name,OUTPUT:result,CONFIDENCE:result
0,1,4611844,Miyagi & Эндшпиль,Saloon,108315318,В салуне злого вестерна я заливал печаль\nНас ...,808778,False,False,False,False,NaN,NaN,NaN
1,2,4611844,Miyagi & Эндшпиль,Silhouette,108315330,О-уо-о-уо\nОу-оу-оу-о-о\n\nКрадётся лёгкий сил...,852296,False,False,False,False,Miyagi & Эндшпиль.Silhouette,no,99.99%
2,3,15691816,ANNA ASTI,Ночью на кухне,108205536,"Она не боится темноты, как в детстве\nБольше н...",961791,False,False,False,False,NaN,NaN,NaN
3,4,3719594,Jah Khalib,Осенняя,108284286,В сером небе птицы улетят\nЗастучит опять что-...,441685,False,False,False,False,NaN,NaN,NaN
4,5,6098375,GOARTUR,Иностранец,107480583,Подари мне первый танец\nЗабери меня с собой\n...,643184,False,False,False,False,GOARTUR.Иностранец,yes,100.00%


In [41]:
final_train = train_df.copy()

In [42]:
final_train.columns

Index(['rank', 'artistId', 'artistsName', 'trackTitle', 'track_id', 'text',
       'rating', 'rating_nan', 'title_regex_flag', 'other_titles_regex_flag',
       'version_regex_flag', 'INPUT:name', 'OUTPUT:result',
       'CONFIDENCE:result'],
      dtype='object')

In [43]:
final_train = final_train[['track_id', 'artistsName', 'trackTitle', 'text', 'OUTPUT:result']]
final_train.drop_duplicates(inplace=True)
print(final_train.info())
final_train.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 0 to 1999
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   track_id       2000 non-null   int64 
 1   artistsName    1997 non-null   object
 2   trackTitle     1997 non-null   object
 3   text           1446 non-null   object
 4   OUTPUT:result  1992 non-null   object
dtypes: int64(1), object(4)
memory usage: 93.8+ KB
None


,track_id,artistsName,trackTitle,text,OUTPUT:result
0,108315318,Miyagi & Эндшпиль,Saloon,В салуне злого вестерна я заливал печаль\nНас ...,NaN
1,108315330,Miyagi & Эндшпиль,Silhouette,О-уо-о-уо\nОу-оу-оу-о-о\n\nКрадётся лёгкий сил...,no
2,108205536,ANNA ASTI,Ночью на кухне,"Она не боится темноты, как в детстве\nБольше н...",NaN
3,108284286,Jah Khalib,Осенняя,В сером небе птицы улетят\nЗастучит опять что-...,NaN
4,107480583,GOARTUR,Иностранец,Подари мне первый танец\nЗабери меня с собой\n...,yes


In [44]:
def get_binary_result(result):
    if result == 'yes':
        return 1
    if result == 'no':
        return 0
    else:
        return -1

In [45]:
final_train['isCover'] = final_train['OUTPUT:result'].apply(get_binary_result)

In [46]:
final_train['OUTPUT:result'].value_counts()

no     1743
yes     249
Name: OUTPUT:result, dtype: int64

In [47]:
final_train

,track_id,artistsName,trackTitle,text,OUTPUT:result,isCover
0,108315318,Miyagi & Эндшпиль,Saloon,В салуне злого вестерна я заливал печаль\nНас ...,NaN,-1
1,108315330,Miyagi & Эндшпиль,Silhouette,О-уо-о-уо\nОу-оу-оу-о-о\n\nКрадётся лёгкий сил...,no,0
2,108205536,ANNA ASTI,Ночью на кухне,"Она не боится темноты, как в детстве\nБольше н...",NaN,-1
3,108284286,Jah Khalib,Осенняя,В сером небе птицы улетят\nЗастучит опять что-...,NaN,-1
4,107480583,GOARTUR,Иностранец,Подари мне первый танец\nЗабери меня с собой\n...,yes,1
...,...,...,...,...,...,...
1995,104956980,Bonsai,Грела сердце,Дева грела сердце человеку у костра\nКрепко об...,no,0
1996,22408881,Kaleida,Think,"Go ahead, you're taking me down now\nGive me, ...",no,0
1997,84608215,GraveChill,Twilight,NaN,no,0
1998,64641986,Lav,Wavvy,I know you like me in the dark\nBut when my ca...,no,0


In [48]:
final_train['artistsName'] = final_train['artistsName'].fillna('')
final_train['trackTitle'] = final_train['trackTitle'].fillna('')

In [49]:
final_train = final_train[(final_train['artistsName'] != '') & (final_train['trackTitle'] != '')]

In [50]:
final_train

,track_id,artistsName,trackTitle,text,OUTPUT:result,isCover
0,108315318,Miyagi & Эндшпиль,Saloon,В салуне злого вестерна я заливал печаль\nНас ...,NaN,-1
1,108315330,Miyagi & Эндшпиль,Silhouette,О-уо-о-уо\nОу-оу-оу-о-о\n\nКрадётся лёгкий сил...,no,0
2,108205536,ANNA ASTI,Ночью на кухне,"Она не боится темноты, как в детстве\nБольше н...",NaN,-1
3,108284286,Jah Khalib,Осенняя,В сером небе птицы улетят\nЗастучит опять что-...,NaN,-1
4,107480583,GOARTUR,Иностранец,Подари мне первый танец\nЗабери меня с собой\n...,yes,1
...,...,...,...,...,...,...
1995,104956980,Bonsai,Грела сердце,Дева грела сердце человеку у костра\nКрепко об...,no,0
1996,22408881,Kaleida,Think,"Go ahead, you're taking me down now\nGive me, ...",no,0
1997,84608215,GraveChill,Twilight,NaN,no,0
1998,64641986,Lav,Wavvy,I know you like me in the dark\nBut when my ca...,no,0


In [51]:
del final_train['OUTPUT:result']

In [52]:
prom = final_train[final_train['isCover'] == -1]
prom

,track_id,artistsName,trackTitle,text,isCover
0,108315318,Miyagi & Эндшпиль,Saloon,В салуне злого вестерна я заливал печаль\nНас ...,-1
2,108205536,ANNA ASTI,Ночью на кухне,"Она не боится темноты, как в детстве\nБольше н...",-1
3,108284286,Jah Khalib,Осенняя,В сером небе птицы улетят\nЗастучит опять что-...,-1
6,107948955,GAYAZOV$ BROTHER$,ФАИНА,"Ай, где же счастье, словно талый снег?\nГде ж,...",-1
42,107058193,NЮ,Некуда бежать,"И бежать мне больше некуда, не к кому, незачем...",-1


In [53]:
final_train['isCover'][0] = 0
final_train['isCover'][2] = 0
final_train['isCover'][3] = 0
final_train['isCover'][6] = 1
final_train['isCover'][42] = 0
final_train

C:\Users\uvd20\AppData\Local\Temp\ipykernel_1308\4255638038.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_train['isCover'][0] = 0
C:\Users\uvd20\AppData\Local\Temp\ipykernel_1308\4255638038.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_train['isCover'][2] = 0
C:\Users\uvd20\AppData\Local\Temp\ipykernel_1308\4255638038.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_train['isCover'][3] = 0
C:\Users\uvd2

,track_id,artistsName,trackTitle,text,isCover
0,108315318,Miyagi & Эндшпиль,Saloon,В салуне злого вестерна я заливал печаль\nНас ...,0
1,108315330,Miyagi & Эндшпиль,Silhouette,О-уо-о-уо\nОу-оу-оу-о-о\n\nКрадётся лёгкий сил...,0
2,108205536,ANNA ASTI,Ночью на кухне,"Она не боится темноты, как в детстве\nБольше н...",0
3,108284286,Jah Khalib,Осенняя,В сером небе птицы улетят\nЗастучит опять что-...,0
4,107480583,GOARTUR,Иностранец,Подари мне первый танец\nЗабери меня с собой\n...,1
...,...,...,...,...,...
1995,104956980,Bonsai,Грела сердце,Дева грела сердце человеку у костра\nКрепко об...,0
1996,22408881,Kaleida,Think,"Go ahead, you're taking me down now\nGive me, ...",0
1997,84608215,GraveChill,Twilight,NaN,0
1998,64641986,Lav,Wavvy,I know you like me in the dark\nBut when my ca...,0


In [54]:
final_train = final_train.merge(df_meta, left_on='track_id', right_on='track_id', how='inner')
print(final_train.info())
final_train.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1997 entries, 0 to 1996
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   track_id                 1997 non-null   int64 
 1   artistsName              1997 non-null   object
 2   trackTitle               1997 non-null   object
 3   text                     1445 non-null   object
 4   isCover                  1997 non-null   int64 
 5   rating                   1997 non-null   int64 
 6   rating_nan               1997 non-null   bool  
 7   title_regex_flag         1997 non-null   bool  
 8   other_titles_regex_flag  1997 non-null   bool  
 9   version_regex_flag       1997 non-null   bool  
dtypes: bool(4), int64(3), object(3)
memory usage: 117.0+ KB
None


,track_id,artistsName,trackTitle,text,isCover,rating,rating_nan,title_regex_flag,other_titles_regex_flag,version_regex_flag
0,108315318,Miyagi & Эндшпиль,Saloon,В салуне злого вестерна я заливал печаль\nНас ...,0,808778,False,False,False,False
1,108315330,Miyagi & Эндшпиль,Silhouette,О-уо-о-уо\nОу-оу-оу-о-о\n\nКрадётся лёгкий сил...,0,852296,False,False,False,False
2,108205536,ANNA ASTI,Ночью на кухне,"Она не боится темноты, как в детстве\nБольше н...",0,961791,False,False,False,False
3,108284286,Jah Khalib,Осенняя,В сером небе птицы улетят\nЗастучит опять что-...,0,441685,False,False,False,False
4,107480583,GOARTUR,Иностранец,Подари мне первый танец\nЗабери меня с собой\n...,1,643184,False,False,False,False


In [55]:
final_train.to_csv('train.csv', index=False)